In [1]:
import tensorflow
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, Masking
import numpy as np
import os
import re
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

# 1. 데이터 전처리

In [2]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 197463


,eng,fra,cc
182692,"Once you have made a promise, you should keep it.","Une fois que vous avez fait une promesse, vous...",CC-BY 2.0 (France) Attribution: tatoeba.org #2...
186910,"I don't know what it is, but it's something ve...","Je ne sais pas de quoi il s'agit, mais c'est q...",CC-BY 2.0 (France) Attribution: tatoeba.org #2...
24194,I love you anyway.,Je t'aime quand même.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
87297,I gave you what you wanted.,Je vous ai donné ce que vous vouliez.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
102629,I had a terrible stomachache.,J'avais un terrible mal d'estomac.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


In [3]:
lines = lines[['eng', 'fra']][:33000] # 3만3천개 샘플 사용
lines.sample(5)

,eng,fra
29129,He is my classmate.,C'est mon camarade de classe.
25161,It's cloudy today.,C'est nuageux aujourd'hui.
5884,That is mine.,C'est à moi.
32362,This is irrelevant.,C'est hors sujet.
51,Hello!,Salut !


In [4]:
# 시작 토큰과 종료 토큰 추가

sos_token = '<start>'
eos_token = '<end>'

def preprocessing_sentence (sentence):
    # Punctuation
    sentence = sentence.lower().strip()
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    
    sentence = sentence.strip()
    sentence = sentence.split(" ")
    
    return sentence
                      
        

In [5]:
def preprocessing_sentence_decoder (sentence):
    sentence = sentence.lower().strip()
  
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!]+", " ", sentence)
    
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    sentence = sentence.split(" ")
    
    return sentence

In [6]:
lines.eng = lines.eng.apply(lambda x : preprocessing_sentence(x))
lines.fra = lines.fra.apply(lambda x : preprocessing_sentence_decoder(x))

In [7]:
eng_tokenizer = Tokenizer()   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # 33000개의 행을 가진 eng의 각 행에 토큰화를 수행

In [8]:
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[28, 1], [28, 1], [28, 1]]

In [9]:
fra_tokenizer = Tokenizer()   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 33000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[1, 75, 8, 2], [1, 365, 3, 2], [1, 28, 512, 8, 2]]

In [10]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4671
프랑스어 단어장의 크기 : 7454


In [11]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [12]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4671
프랑스어 단어장의 크기 : 7454
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 17


In [13]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]


In [14]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 75, 8], [1, 365, 3], [1, 28, 512, 8]]
[[75, 8, 2], [365, 3, 2], [28, 512, 8, 2]]


In [15]:
# padding
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [16]:
# encoder_input = to_categorical(encoder_input)
# decoder_input = to_categorical(decoder_input)
# decoder_target = to_categorical(decoder_target)
# print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
# print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
# print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

In [17]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [18]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (33000, 8)
프랑스어 학습 입력데이터의 크기(shape) : (33000, 17)
프랑스어 학습 출력데이터의 크기(shape) : (33000, 17)


# 2. 모델 훈련

In [19]:
embedding_state = 256
hidden_state = 256

In [20]:
# 인코더에서 사용할 임베딩
encoder_inputs = Input(shape=(None, ), name='encoder_input')
enc_emb =  Embedding(eng_vocab_size, embedding_state, input_length=max_eng_seq_len)(encoder_inputs)
encoder_lstm = LSTM(units = 256, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [21]:
# 입력 decoder
decoder_inputs = Input(shape=(None, ), name='decoder_input')
dec_emb =  Embedding(fra_vocab_size, embedding_state)(decoder_inputs)
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
decoder_outputs, state_h, state_c = decoder_lstm(dec_emb, initial_state = encoder_states)

In [22]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [23]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics = ['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    1195776     encoder_input[0][0]              
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    1908224     decoder_input[0][0]              
______________________________________________________________________________________________

In [44]:
model.fit(x=[encoder_input_train, decoder_input_train], 
          y=decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=30, epochs=50)

Epoch 1/50
1000/1000 [==============================] - 11s 11ms/step - loss: 0.5178 - acc: 0.9290 - val_loss: 0.8062 - val_acc: 0.8799
Epoch 2/50
1000/1000 [==============================] - 11s 11ms/step - loss: 0.5139 - acc: 0.9291 - val_loss: 0.8080 - val_acc: 0.8802
Epoch 3/50
1000/1000 [==============================] - 10s 10ms/step - loss: 0.5111 - acc: 0.9299 - val_loss: 0.8047 - val_acc: 0.8804
Epoch 4/50
1000/1000 [==============================] - 10s 10ms/step - loss: 0.5073 - acc: 0.9302 - val_loss: 0.8075 - val_acc: 0.8787
Epoch 5/50
1000/1000 [==============================] - 10s 10ms/step - loss: 0.5032 - acc: 0.9307 - val_loss: 0.8053 - val_acc: 0.8806
Epoch 6/50
1000/1000 [==============================] - 10s 10ms/step - loss: 0.5001 - acc: 0.9312 - val_loss: 0.8067 - val_acc: 0.8796
Epoch 7/50
1000/1000 [==============================] - 10s 10ms/step - loss: 0.4974 - acc: 0.9314 - val_loss: 0.8066 - val_acc: 0.8800
Epoch 8/50
1000/1000 [==========================

# 3. 모델 테스트하기

In [45]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         1195776   
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 525312    
Total params: 1,721,088
Trainable params: 1,721,088
Non-trainable params: 0
_________________________________________________________________


In [46]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = Embedding(fra_vocab_size, 256)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]


In [47]:
decoder_outputs = decoder_softmax_layer(decoder_outputs2)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 256)    1908224     decoder_input[0][0]              
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
____________________________________________________________________________________________

In [48]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [49]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [50]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [51]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<start>']) and i!=fra2idx['<end>']):
            temp = temp + idx2fra[i] + ' '
    return temp

In [52]:
for seq_index in [3,77,531,733,2373]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index])) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: i m so tired . 
정답 문장: je suis tellement fatigu . 
번역기가 번역한 문장:  sens y les les le
-----------------------------------
입력 문장: tom intervened . 
정답 문장: tom intervint . 
번역기가 번역한 문장:  on on mauvais lu
-----------------------------------
입력 문장: are you guys ok ? 
정답 문장: a va tout le monde ? 
번역기가 번역한 문장:  on tellement tellemen
-----------------------------------
입력 문장: we re rich . 
정답 문장: nous sommes riches . 
번역기가 번역한 문장:  avez vrai te te t
-----------------------------------
입력 문장: who plays golf ? 
정답 문장: qui joue au golf ? 
번역기가 번역한 문장:  manger dis dis di
